In [ ]:
import glob
import os

import pandas as pd
pd.set_option('display.max_colwidth', 255)

from dpp_helpline_qa.model_validation.model_validation import cal_em_score, calculate_semantic_similarity
from dpp_helpline_qa.modelling.question_answer import load_model_flan, answer_question_flan
from dpp_helpline_qa.modelling.semantic_search import load_model_ss, context_ranking
from dpp_helpline_qa.preprocessing.preprocessing import process_docs

In [ ]:
# list of questions to ask
questions = [
"My test counts have identified differences between the physical number of items in stock and the number recorded in the accounts. Does this mean that I can't place any reliance on the stock count as a control?",
    "Do I need to attend an entity's inventory count?",
    "How do I audit inventory if it is held by a third-party?",
    "When is it impracticable to attend an inventory count?",
    "Do we need to perform procedures to determine whether the entity's final inventory records accurately reflect the actual inventory count results? If so, what procedures do we perform?",
    "what procedures do we need to perform to determine whether the entity's final inventory records accurately reflect the actual inventory count results?",
    "What do you need to document for materiality?",
    "When in the audit do I determine materiality?",
    "What factors affect the amount of aggregation risk used when setting performance materiality?"
]

In [ ]:
question_no = 5
# semantically search the documents and use the most similar section of text to generate an answer to the question
question = questions[question_no]
op_files = glob.glob('../output/Materiality/*.csv') #Inventory
context_df = context_ranking(question, op_files, model_ss, tokenizer_ss)
main_context = '\n'.join(context_df['content'].values[0:5])
output = answer_question(model_qa, tokenizer_qa, main_context, question)

In [ ]:
# sectioning of doc
import fitz
from dpp_helpline_qa.preprocessing.preprocessing import fonts, font_tags, headers_para, clean_doc, break_into_para

doc1 = os.path.join('..', "data", "Materiality", "KAEG part 4" + ".pdf")
doc = fitz.open(doc1)
doc_name = os.path.basename(doc.name)
font_counts, styles = fonts(doc)
size_tag, p_style = font_tags(doc_name, font_counts, styles)
elements = headers_para(doc, size_tag, p_style)
text_lines = clean_doc(elements)
new_para = break_into_para(text_lines, doc_name)
print(len(new_para))
new_para